In [1]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import random
import re
import builtin.utils as ut

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 3, Finished, Available, Finished)

In [2]:

# Mount external lakehouse  "Bronze" into specified local path.
notebookutils.fs.mount("abfss://8c7d2edc-a6a8-4f1a-bef1-c8b14449cfd3@onelake.dfs.fabric.microsoft.com/a1b6b3d0-9bef-4e2a-a731-81e4d801d7a1", "/lakehouse/Bronze")

import pandas as pd
# Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/Bronze/Files/Data_Parquet/user_reviews.parquet')
df = pd.read_parquet(notebookutils.fs.getMountPath('/lakehouse/Bronze/Files/Data_Parquet/user_reviews.parquet'))
display(df)


StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a3db3efb-b4b7-4844-919d-5ceed5db20a1)

SynapseWidget(Synapse.DataFrame, b0395ed4-f784-4036-8f1a-0ba34675ac16)

In [3]:
ut.data_summ(df, title=None)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 5, Finished, Available, Finished)

,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,user_id,[<class 'str'>],25799,0.0,0
1,user_url,[<class 'str'>],25799,0.0,0
2,reviews,[<class 'numpy.ndarray'>],25799,0.0,0


We going to analize duplicate rows by user

In [4]:
duplicated = ut.duplicates(df, 'user_id')
duplicated

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 6, Finished, Available, Finished)

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'helpful': '4 of 4 people (100%..."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'helpful': '4 of 4 people (100%..."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'helpful': '5 of 7 people (71%)..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'helpful': '5 of 7 people (71%)..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'helpful': '2 of 2 people (100%..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'helpful': '2 of 5 people (40%)..."


In [5]:
df = ut.drop_duplicates(df, 'user_id')

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 7, Finished, Available, Finished)

In [6]:
df.shape

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 8, Finished, Available, Finished)

(25485, 3)

Now, We are going to expand the data that is nested in the reviews column

In [7]:
unpacked_data = []
for index, row in df.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    for review_dict in row['reviews']:
        review_dict['user_id'] = user_id
        review_dict['user_url'] = user_url
        unpacked_data.append(review_dict)

df = pd.DataFrame(unpacked_data)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 9, Finished, Available, Finished)

In [8]:
ut.data_summ(df)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 10, Finished, Available, Finished)

,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,funny,[<class 'str'>],58430,0.0,0
1,helpful,[<class 'str'>],58430,0.0,0
2,item_id,[<class 'str'>],58430,0.0,0
3,last_edited,[<class 'str'>],58430,0.0,0
4,posted,[<class 'str'>],58430,0.0,0
5,recommend,[<class 'bool'>],58430,0.0,0
6,review,[<class 'str'>],58430,0.0,0
7,user_id,[<class 'str'>],58430,0.0,0
8,user_url,[<class 'str'>],58430,0.0,0


In [9]:
df.replace('', None, inplace=True)
df.head()

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 11, Finished, Available, Finished)

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id,user_url
0,None,No ratings yet,1250,None,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,None,No ratings yet,22200,None,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,None,No ratings yet,43110,None,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,None,15 of 20 people (75%) found this review helpful,251610,None,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,None,0 of 1 people (0%) found this review helpful,227300,None,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


In [10]:
ut.replace_all_nulls(df)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 12, Finished, Available, Finished)

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id,user_url
0,No data,No ratings yet,1250,No data,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,No data,No ratings yet,22200,No data,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,No data,No ratings yet,43110,No data,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,No data,15 of 20 people (75%) found this review helpful,251610,No data,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,No data,0 of 1 people (0%) found this review helpful,227300,No data,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
58425,No data,No ratings yet,70,No data,Posted July 10.,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58426,No data,No ratings yet,362890,No data,Posted July 8.,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58427,1 person found this review funny,1 of 2 people (50%) found this review helpful,273110,No data,Posted July 3.,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
58428,No data,No ratings yet,730,No data,Posted July 20.,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In order to clean the dataframe focused in our objective, We going to extarct year in column 'posted'

In [11]:
def date_post(date_str):
    date_r = re.search(r'(\w+\s\d{1,2},\s\d{4})', date_str)
    if date_r:
        date_rvw = date_r.group(1)
        try:
            date_rt = pd.to_datetime(date_rvw)
            return date_rt.strftime('%Y')
        except ValueError:
            return 'Invalid Date'
    else:
        return 'Invalid Format'

df['posted'] = df['posted'].apply(date_post)
df = df[df['posted'] != 'Invalid Format']
df['posted'] 

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 13, Finished, Available, Finished)

0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
58377    2015
58380    2015
58390    2015
58392    2015
58401    2015
Name: posted, Length: 48498, dtype: object

In [12]:
df = df.drop(["funny", "last_edited", "helpful", "user_url"], axis=1)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 14, Finished, Available, Finished)

In [13]:
new_names = {'item_id': 'Item_id', 'posted': 'Posted', 'recommend': 'Recommend', 'review': 'Review', 'user_id': 'User_id'}

# Use the rename() method to change column names
df.rename(columns=new_names, inplace=True)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 15, Finished, Available, Finished)

In [14]:
df['Item_id'] = df['Item_id'].fillna(0).astype(int)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 16, Finished, Available, Finished)

In [15]:
df.head()

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 17, Finished, Available, Finished)

,Item_id,Posted,Recommend,Review,User_id
0,1250,2011,True,Simple yet with great replayability. In my opi...,76561197970982479
1,22200,2011,True,It's unique and worth a playthrough.,76561197970982479
2,43110,2011,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,251610,2014,True,I know what you think when you see this title ...,js41637
4,227300,2013,True,For a simple (it's actually not all that simpl...,js41637


In [16]:
archivo_parquet = '/lakehouse/default/Files/Cleaned_Data/ureviews.parquet'

# Export to a parquet file.
df.to_parquet(archivo_parquet, index=False)

StatementMeta(, 8f6721a1-eec9-4d25-ba74-5d9623cec177, 18, Finished, Available, Finished)